In [16]:
!docker pull osrm/osrm-backend
!wget http://download.geofabrik.de/europe/germany/berlin-latest.osm.pbf
!docker run -t -v $(pwd):/data osrm/osrm-backend osrm-extract -p /opt/car.lua /data/berlin-latest.osm.pbf
!docker run -t -v $(pwd):/data osrm/osrm-backend osrm-partition /data/berlin-latest.osrm
!docker run -t -v $(pwd):/data osrm/osrm-backend osrm-customize /data/berlin-latest.osrm
!docker run -d -p 5000:5000 -v $(pwd):/data osrm/osrm-backend osrm-routed --algorithm mld /data/berlin-latest.osrm

Using default tag: latest
latest: Pulling from osrm/osrm-backend
Digest: sha256:af5d4a83fb90086a43b1ae2ca22872e6768766ad5fcbb07a29ff90ec644ee409
Status: Image is up to date for osrm/osrm-backend:latest
docker.io/osrm/osrm-backend:latest
--2024-07-08 09:55:26--  http://download.geofabrik.de/europe/germany/berlin-latest.osm.pbf
Resolving download.geofabrik.de (download.geofabrik.de)... 65.109.50.43, 65.109.48.72, 2a01:4f9:5a:2797::2, ...
Connecting to download.geofabrik.de (download.geofabrik.de)|65.109.50.43|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85303820 (81M) [application/octet-stream]
Saving to: ‘berlin-latest.osm.pbf.3’

berlin-latest.osm.p 100%[===================>]  81.35M  14.8MB/s    in 6.6s    

2024-07-08 09:55:34 (12.3 MB/s) - ‘berlin-latest.osm.pbf.3’ saved [85303820/85303820]

[info] Parsed 0 location-dependent features with 0 GeoJSON polygons
[info] Using script /opt/car.lua
[info] Input file: berlin-latest.osm.pbf
[info] Profile: car.lua


In [17]:
import pandas as pd
import numpy as np

df_dp = pd.read_csv("/workspaces/Accident-Opt-Strabag/DataFiles/KMeans_clusters.csv")
df_s=pd.read_csv("/workspaces/Accident-Opt-Strabag/DataFiles/sources_coord.csv")
# Drop the first row if needed

In [18]:
# Extract coordinates from the DataFrame
coordinates_dp = df_dp[['Accident_LATITUDE', 'Accident_LONGITUDE']]
coordinates_s=df_s[['Latitude','Longitude']]

# Initialize an empty distance matrix
num_rows = len(df_dp)

In [58]:
import requests

# Replace with your actual API key
api_key = '5b3ce3597851110001cf62484fb7a46e7d57426cbaaed9638ce98d0b'

# Example coordinates (longitude, latitude)

def get_distance_each_dp(dp_lat,dp_lon,s):
        # Replace with your actual API key
        api_key = '5b3ce3597851110001cf62484fb7a46e7d57426cbaaed9638ce98d0b'
        dp_cord=[]
        distance_list=[]
        coordinates=[]
        dp_cord.append(dp_lat)
        dp_cord.append(dp_lon)
        coordinates.append(dp_cord)
         # ORSM API endpoint for matrix
        url = 'https://api.openrouteservice.org/v2/matrix/driving-car'
        for j in range(len(s)):

            s_lat=s.loc[j,'Latitude']
            s_lon=s.loc[j,'Longitude']
            s_cord=[]
            s_cord.append(s_lat)
            s_cord.append(s_lon)
            coordinates.append(s_cord)
            #coordinates=coordinates.tolist()

        body = {"locations":coordinates}

        headers = {
            'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
            'Authorization': '5b3ce3597851110001cf62484fb7a46e7d57426cbaaed9638ce98d0b',
            'Content-Type': 'application/json; charset=utf-8'
        }
        response = requests.post(url, json=body, headers=headers)
        print(response.text)

        # Check if request was successful
        if response.status_code == 200:
            data = response.json()
            print(data)

            # Extract distance between the two points (in meters)
            distance_meters = data['distances'][0][1]  # Assuming you want distance from point 1 to point 2
            distance_list.append(distance_meters)

            print(f"Distance between points: {distance_meters} meters")
        else:
            print(f"Error: {response.status_code}, {response.text}")
        print(distance_list)
        return distance_list


In [59]:
df=pd.DataFrame()
for i in range(len(df_dp)):
    lat=df_dp['Accident_LATITUDE'].iloc[i]
    lon=df_dp['Accident_LONGITUDE'].iloc[i]

    distance_listfor_i=get_distance_each_dp(lat,lon,df_s)
    print(distance_listfor_i)
    df[f'dp_{i}-s']=distance_listfor_i



{"durations":[[null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null],[null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null],[null,null,0.0,415.96,null,null,null,null,null,null,null,null,null,null,null,null,null],[null,null,415.96,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null],[null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null],[null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null],[null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null],[null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null],[null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null],[null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null],[null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null],[null,null,null,null,null,n

KeyError: 'distances'